In [3]:
import pickle
import pandas as pd
import numpy as np

In [41]:
def isoform_in_window(starts, w_s, w_e):
    #print(starts, w_s, w_e)
    return any([x >= w_s and x <= w_e for x in starts])

In [ ]:
def distance_from_isoform(starts, w_s, w_e):
    for 

In [4]:
np.random.seed(73298)

In [24]:
generated_dataset = pickle.load(open("second_dataset.pickle", "rb"))

In [25]:
generated_dataset = [x for x in generated_dataset if max(x['density']) > 10]

In [26]:
generated_dataset[:100]

[{'chrom': 'chrI',
  'strand': '+',
  'gene': 'YAL066W',
  'isoforms': [<simulate_reads.Isoform at 0x7f11bdfe2470>,
  'distr': array([0.25600663, 0.13504557, 0.6089478 ]),
  'density': [61,
   125,
   183,
   247,
   301,
   361,
   421,
   485,
   549,
   609,
   670,
   727,
   778,
   841,
   895,
   960,
   1020,
   1079,
   1141,
   1211,
   1266,
   1343,
   1391,
   1443,
   1500,
   1563,
   1628,
   1685,
   1747,
   1815,
   1882,
   1937,
   2013,
   2056,
   2122,
   2195,
   2250,
   2318,
   2381,
   2449,
   2504,
   2578,
   2641,
   2706,
   2768,
   2766,
   2769,
   2770,
   2783,
   2810,
   2819,
   2831,
   2819,
   2823,
   2826,
   2830,
   2832,
   2854,
   2854,
   2868,
   2886,
   2887,
   2909,
   2915,
   2927,
   2961,
   2961,
   2985,
   3011,
   3019,
   3022,
   3023,
   3046,
   3066,
   3086,
   3095,
   3101,
   3108,
   3148,
   3163,
   3160,
   3178,
   3188,
   3208,
   3218,
   3255,
   3281,
   3294,
   3300,
   3297,
   3315,
   3322,
   333

In [8]:
import helper
genome = helper.organize_genome_by_chrom(helper.read_bedfile())

INFO:helper:Reading bedfile...
INFO:helper:Done reading bedfile ../indexes/yeast-all.bed


In [55]:
window = 50
colnames = [str(i) for i in range(-window, window+1)] + ['Isoform']

labeled_data = []

keep_non_isoforms_threshold = 1

In [56]:
for entry in generated_dataset: 
    isoform_starts = [x.start for x in entry['isoforms']]
    isoform_starts = sorted(isoform_starts)
    gene_start = isoform_starts[0]
    utr_end = int(genome[entry['chrom']][entry['gene']].fields[6])
    utr_end = min(utr_end, isoform_starts[-1])
    for pos in range(gene_start + window, utr_end - window + 1):
        ws = (pos-window) - gene_start # density array is indexed from 0. pos 0 is gene start
        we = (pos+window+1) - gene_start 
        
        isoform =  int(isoform_in_window(isoform_starts, pos-window, pos+window+1))
        if isoform == 0:
            if np.random.uniform() > keep_non_isoforms_threshold:
                continue
        row = entry['density'][ws:we] + [isoform]
    
        labeled_data.append(dict(zip(colnames, row)))

In [58]:
labeled_data = pd.DataFrame(labeled_data)
labeled_data['-50'].describe()

count    159872.000000
mean         18.472528
std          93.493307
min           0.000000
25%           1.000000
50%           4.000000
75%          11.000000
max        3927.000000
Name: -50, dtype: float64

In [59]:
labeled_data['Isoform'].value_counts()

0    82050
1    77822
Name: Isoform, dtype: int64

In [60]:
from sklearn.model_selection import train_test_split

In [3]:
y = labeled_data.pop('Isoform')
X = labeled_data

NameError: name 'labeled_data' is not defined

In [62]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [63]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
classifier = Sequential()
classifier.add(Dense(units = 101, activation='sigmoid', input_dim=101))
classifier.add(Dense(units = 202, activation='relu'))
classifier.add(Dense(units = 102, activation='relu'))
classifier.add(Dense(units = 1, activation='sigmoid'))
classifier.compile(optimizer='sgd', loss='binary_crossentropy')



NameError: name 'Sequential' is not defined

In [1]:
classifier.fit(X_train, y_train, batch_size=1, epochs=30)

NameError: name 'classifier' is not defined

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = [1 if Y >= 0.9 else 0 for Y in y_pred]


In [ ]:
for i in range(0, len(y_pred)):
    if int(y_pred[i]) == 1 and int(y_test.iloc[i]) == 1:
        print(X_test.iloc[i])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)